In [29]:
# Parses the rollout data from demo.py and saves them as numpy arrays of states, actions, and pot handle positions (the conditional vector data)

import numpy as np
import csv
import matplotlib.pyplot as plt
import pickle as pkl
from scipy.spatial.transform import Rotation as R
from transform_utils import quat_to_rot6d, rotvec_to_rot6d, rot6d_to_quat


In [6]:
# with open("rollouts_pot/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])
#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
#     state = np.hstack([state, robot0_eef_rotvec])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
#     state = np.hstack([state, robot1_eef_rotvec])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])

# print(np.shape(state))
# print(np.shape(actions))

In [ ]:
expert_states_list = []
expert_actions_list = []
pot_start_list = []
pot_states_list1 = []
pot_states_list2 = []
for j in range(50):
    for i in [2, 3]:
# for i in [2, 3]:
#     for j in range(50):   
        with open("rollouts/newslower_closegrip/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            print(np.shape(actions))
            pot1 = np.array(rollout["pot_states1"])
            pot2 = np.array(rollout["pot_states2"])
            pot = np.array(rollout["pot_start"])

            pot_start_list.append(np.concatenate((pot[0], pot[1])))

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 900 - actions.shape[0]

            try:

                repeated_last = np.tile(actions[-1], (repeats_needed, 1))
                actions = np.vstack([actions, repeated_last])

                repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
                robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
                state = robot0_eef_pos

                repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
                robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
                robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
                state = np.hstack([state, robot0_eef_rotvec])


                repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
                robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
                robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
                state = np.hstack([state, robot0_gripper_pos])

                repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
                robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
                state = np.hstack([state, robot1_eef_pos])

                repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
                robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
                robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
                state = np.hstack([state, robot1_eef_rotvec])

                repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
                robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
                robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
                state = np.hstack([state, robot1_gripper_pos])

                repeated_last = np.tile(pot1[-1], (repeats_needed, 1))
                pot1 = np.vstack([pot1, repeated_last])
                repeated_last = np.tile(pot2[-1], (repeats_needed, 1))
                pot2 = np.vstack([pot2, repeated_last])
                pot_states_list1.append(pot1)
                pot_states_list2.append(pot2)

                expert_states_list.append(state)
                expert_actions_list.append(actions)
            except:
                print("error with file seed%s mode%s" % (j*10, i))
                continue

expert_states_rotvec = np.stack(expert_states_list, axis=0)
expert_actions_rotvec = np.stack(expert_actions_list, axis=0)
pot_states_rotvec1 = np.stack(pot_states_list1, axis=0)
pot_states_rotvec2 = np.stack(pot_states_list2, axis=0)
pot_start_rotvec = np.stack(pot_start_list, axis=0)

(821, 14)
(752, 14)
(815, 14)
(855, 14)
(794, 14)
(875, 14)
(798, 14)
(847, 14)
(744, 14)
(827, 14)
(832, 14)
(815, 14)
(860, 14)
(792, 14)
(864, 14)
(791, 14)
(841, 14)
(816, 14)
(829, 14)
(830, 14)
(778, 14)
(875, 14)
(771, 14)
(791, 14)
(775, 14)
(874, 14)
(843, 14)
(829, 14)
(779, 14)
(862, 14)
(807, 14)
(822, 14)
(809, 14)
(799, 14)
(849, 14)
(790, 14)
(808, 14)
(756, 14)
(873, 14)
(800, 14)
(800, 14)
(845, 14)
(787, 14)
(860, 14)
(791, 14)
(855, 14)
(828, 14)
(842, 14)
(852, 14)
(795, 14)
(753, 14)
(832, 14)
(860, 14)
(785, 14)
(836, 14)
(785, 14)
(806, 14)
(856, 14)
(848, 14)
(806, 14)
(825, 14)
(824, 14)
(866, 14)
(792, 14)
(825, 14)
(836, 14)
(855, 14)
(786, 14)
(872, 14)
(792, 14)
(832, 14)
(809, 14)
(848, 14)
(789, 14)
(837, 14)
(791, 14)
(821, 14)
(811, 14)
(752, 14)
(815, 14)
(772, 14)
(870, 14)
(847, 14)
(803, 14)
(1161, 14)
error with file seed420 mode2
(1215, 14)
error with file seed420 mode3
(853, 14)
(809, 14)
(811, 14)
(861, 14)
(789, 14)
(851, 14)
(785, 14)
(848, 14

In [46]:
expert_states_list = []
expert_actions_list = []
pot_start_list = []
pot_states_list1 = []
pot_states_list2 = []

for j in range(50):
    for i in [2, 3]:
        try:
            with open("rollouts/newslower_closegrip_orig/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
                rollout = pkl.load(f)
        except Exception as e:
            print("skip (open): seed%s mode%s — %s" % (j*10, i, e))
            continue

        obs = rollout["observations"]
        actions = np.array(rollout["actions"])
        pot1 = np.array(rollout["pot_states1"])
        pot2 = np.array(rollout["pot_states2"])
        pot = np.array(rollout["pot_start"])

        # --- minimal validity checks (no trimming) ---
        T_actions = actions.shape[0]
        if T_actions > 900:
            print("skip (>900): seed%s mode%s — actions=%d" % (j*10, i, T_actions))
            continue

        # Build streams that must match actions length
        robot0_eef_pos   = np.array([o["robot0_eef_pos"]        for o in obs])
        robot0_eef_quat  = np.array([o["robot0_eef_quat_site"]  for o in obs])
        robot0_gripper_pos = np.array([o["robot0_gripper_pos"]  for o in obs])
        robot1_eef_pos   = np.array([o["robot1_eef_pos"]        for o in obs])
        robot1_eef_quat  = np.array([o["robot1_eef_quat_site"]  for o in obs])
        robot1_gripper_pos = np.array([o["robot1_gripper_pos"]  for o in obs])

        lengths = [
            T_actions,
            len(obs),
            robot0_eef_pos.shape[0],
            robot0_eef_quat.shape[0],
            robot0_gripper_pos.shape[0],
            robot1_eef_pos.shape[0],
            robot1_eef_quat.shape[0],
            robot1_gripper_pos.shape[0],
            pot1.shape[0],
            pot2.shape[0],
        ]
        if not all(L == T_actions for L in lengths):
            print("skip (length mismatch): seed%s mode%s — %s" % (j*10, i, lengths))
            continue
        # ---------------------------------------------

        repeats_needed = 900 - T_actions

        try:
            # only record pot_start if we pass checks
            pot_start_list.append(np.concatenate((pot[0], pot[1])))

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            robot0_eef_rotvec = R.from_quat(robot0_eef_quat).as_rotvec()
            state = np.hstack([state, robot0_eef_rotvec])

            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            robot1_eef_rotvec = R.from_quat(robot1_eef_quat).as_rotvec()
            state = np.hstack([state, robot1_eef_rotvec])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            repeated_last = np.tile(pot1[-1], (repeats_needed, 1))
            pot1 = np.vstack([pot1, repeated_last])
            repeated_last = np.tile(pot2[-1], (repeats_needed, 1))
            pot2 = np.vstack([pot2, repeated_last])
            pot_states_list1.append(pot1)
            pot_states_list2.append(pot2)

            expert_states_list.append(state)
            expert_actions_list.append(actions)
        except Exception as e:
            print("error with file seed%s mode%s — %s" % (j*10, i, e))
            # If something failed during padding/stacking, roll back the pot_start append
            if len(pot_start_list) > len(expert_states_list):
                pot_start_list.pop()
            continue

expert_states_rotvec = np.stack(expert_states_list, axis=0)
expert_actions_rotvec = np.stack(expert_actions_list, axis=0)
pot_states_rotvec1 = np.stack(pot_states_list1, axis=0)
pot_states_rotvec2 = np.stack(pot_states_list2, axis=0)
pot_start_rotvec = np.stack(pot_start_list, axis=0)


In [47]:
print(np.shape(expert_states_rotvec))
print(np.shape(expert_actions_rotvec))
print(np.shape(pot_states_rotvec1))
print(np.shape(pot_states_rotvec2))
print(np.shape(pot_start_rotvec))

(100, 900, 14)
(100, 900, 14)
(100, 900, 3)
(100, 900, 3)
(100, 6)


In [48]:
np.save("data/expert_states_newslower_100_alternatemode_closegrip_orig.npy", expert_states_rotvec)
np.save("data/expert_actions_newslower_100_alternatemode_closegrip_orig.npy", expert_actions_rotvec)
np.save("data/pot_states1_newslower_100_alternatemode_closegrip_orig.npy", pot_states_rotvec1)
np.save("data/pot_states2_newslower_100_alternatemode_closegrip_orig.npy", pot_states_rotvec2)
np.save("data/pot_start_newslower_100_alternatemode_closegrip_orig.npy", pot_start_rotvec)


In [13]:
# with open("rollouts/rollout_seed0_mode2.pkl", "rb") as f:
#     rollout = pkl.load(f)
#     obs = rollout["observations"]
#     actions = np.array(rollout["actions"])

#     pos0 = actions[:,:3]
#     rotvec0 = actions[:,3:6]
#     gripper0 = actions[:,6]
#     pos1 = actions[:,7:10]
#     rotvec1 = actions[:,10:13]
#     gripper1 = actions[:,13]

#     rot6d_list0 = []
#     for rv in rotvec0:
#         rot6d_list0.append(rotvec_to_rot6d(rv))
#     rot6d0 = np.array(rot6d_list0)

#     rot6d_list1 = []
#     for rv in rotvec1:
#         rot6d_list1.append(rotvec_to_rot6d(rv))
#     rot6d1 = np.array(rot6d_list1)

#     actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

#     robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
#     robot0_eef_quat = np.array([o["robot0_eef_quat"] for o in obs])
#     robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
#     robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
#     robot1_eef_quat = np.array([o["robot1_eef_quat"] for o in obs])
#     robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

#     repeats_needed = 250 - actions.shape[0]

#     repeated_last = np.tile(actions[-1], (repeats_needed, 1))
#     actions = np.vstack([actions, repeated_last])

#     repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
#     robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
#     state = robot0_eef_pos

#     repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
#     robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
#     eef_rot6d0 = []
#     for q in robot0_eef_quat:
#         eef_rot6d0.append(quat_to_rot6d(q))
#     robot0_eef_rot6d = np.array(eef_rot6d0)
#     state = np.hstack([state, robot0_eef_rot6d])


#     repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
#     robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
#     robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
#     state = np.hstack([state, robot0_gripper_pos])

#     repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
#     robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
#     state = np.hstack([state, robot1_eef_pos])

#     repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
#     robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
#     eef_rot6d1 = []
#     for q in robot1_eef_quat:
#         eef_rot6d1.append(quat_to_rot6d(q))
#     robot1_eef_rot6d = np.array(eef_rot6d1)
#     state = np.hstack([state, robot1_eef_rot6d])

#     repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
#     robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
#     robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
#     state = np.hstack([state, robot1_gripper_pos])
    
# print(np.shape(state))
# print(np.shape(actions))

In [15]:
expert_states_list = []
expert_actions_list = []
pot_states_list = []
for i in [2, 3]:
    for j in range(10):   
        with open("rollouts_grippause/rollout_seed%s_mode%s.pkl" % (j*10, i), "rb") as f:
            rollout = pkl.load(f)
            obs = rollout["observations"]
            actions = np.array(rollout["actions"])
            pot = np.array(rollout["pot_pos"])

            pot_states_list.append(np.concatenate((pot[0], pot[1])))

            pos0 = actions[:,:3]
            rotvec0 = actions[:,3:6]
            gripper0 = actions[:,6]
            pos1 = actions[:,7:10]
            rotvec1 = actions[:,10:13]
            gripper1 = actions[:,13]

            rot6d_list0 = []
            for rv in rotvec0:
                rot6d_list0.append(rotvec_to_rot6d(rv))
            rot6d0 = np.array(rot6d_list0)

            rot6d_list1 = []
            for rv in rotvec1:
                rot6d_list1.append(rotvec_to_rot6d(rv))
            rot6d1 = np.array(rot6d_list1)

            actions = np.concatenate((pos0, rot6d0, gripper0.reshape(-1, 1), pos1, rot6d1, gripper1.reshape(-1, 1)), axis=1)

            robot0_eef_pos = np.array([o["robot0_eef_pos"] for o in obs])
            robot0_eef_quat = np.array([o["robot0_eef_quat_site"] for o in obs])
            robot0_gripper_pos = np.array([o["robot0_gripper_pos"] for o in obs])
            robot1_eef_pos = np.array([o["robot1_eef_pos"] for o in obs])
            robot1_eef_quat = np.array([o["robot1_eef_quat_site"] for o in obs])
            robot1_gripper_pos = np.array([o["robot1_gripper_pos"] for o in obs])

            repeats_needed = 400 - actions.shape[0]

            repeated_last = np.tile(actions[-1], (repeats_needed, 1))
            actions = np.vstack([actions, repeated_last])

            repeated_last = np.tile(robot0_eef_pos[-1], (repeats_needed, 1))
            robot0_eef_pos = np.vstack([robot0_eef_pos, repeated_last])
            state = robot0_eef_pos

            repeated_last = np.tile(robot0_eef_quat[-1], (repeats_needed, 1))
            robot0_eef_quat = np.vstack([robot0_eef_quat, repeated_last])
            eef_rot6d0 = []
            for q in robot0_eef_quat:
                eef_rot6d0.append(quat_to_rot6d(q))
            robot0_eef_rot6d = np.array(eef_rot6d0)
            state = np.hstack([state, robot0_eef_rot6d])


            repeated_last = np.tile(robot0_gripper_pos[-1], (repeats_needed, 1))
            robot0_gripper_pos = robot0_gripper_pos.reshape(-1, 1)
            robot0_gripper_pos = np.vstack([robot0_gripper_pos, repeated_last])
            state = np.hstack([state, robot0_gripper_pos])

            repeated_last = np.tile(robot1_eef_pos[-1], (repeats_needed, 1))
            robot1_eef_pos = np.vstack([robot1_eef_pos, repeated_last])
            state = np.hstack([state, robot1_eef_pos])

            repeated_last = np.tile(robot1_eef_quat[-1], (repeats_needed, 1))
            robot1_eef_quat = np.vstack([robot1_eef_quat, repeated_last])
            eef_rot6d1 = []
            for q in robot1_eef_quat:
                eef_rot6d1.append(quat_to_rot6d(q))
            robot1_eef_rot6d = np.array(eef_rot6d1)
            state = np.hstack([state, robot1_eef_rot6d])

            repeated_last = np.tile(robot1_gripper_pos[-1], (repeats_needed, 1))
            robot1_gripper_pos = robot1_gripper_pos.reshape(-1, 1)
            robot1_gripper_pos = np.vstack([robot1_gripper_pos, repeated_last])
            state = np.hstack([state, robot1_gripper_pos])

            expert_states_list.append(state)
            expert_actions_list.append(actions)

expert_states_rot6d = np.stack(expert_states_list, axis=0)
expert_actions_rot6d = np.stack(expert_actions_list, axis=0)
pot_states_rot6d = np.stack(pot_states_list, axis=0)

In [16]:
print(np.shape(expert_states_rot6d))
print(np.shape(expert_actions_rot6d))
print(np.shape(pot_states_rot6d))

(20, 400, 20)
(20, 400, 20)
(20, 6)


In [17]:
np.save("data/expert_states_rot6d_site_grippause_20.npy", expert_states_rot6d)
np.save("data/expert_actions_rot6d_site_grippause_20.npy", expert_actions_rot6d)
np.save("data/pot_states_rot6d_site_grippause_20.npy", pot_states_rot6d)